In [46]:
# Import librarais Projet SID 21
# bibliothéque basique
import numpy as np 
import pandas as pd
# bibliotéque connection oracle database

# bibliothéque dash
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
#### pour les graphes
import plotly.graph_objects as go
from plotly.subplots import make_subplots
### word cloud
from wordcloud import WordCloud, STOPWORDS
import plotly.graph_objs as go

# Partie I) connection avec oracle et récuperation des données.

In [ ]:
## oracle

In [45]:
# import data
df=pd.read_csv('/home/sid2020-7/M1 S2/Projet Dashbord/Dashboard_V7.csv')

# Partie I I ) Nettoyage des données.

In [47]:

### fonction pour netoyer le colonne keywords
def keywords(txt):
    b = []
    txt=str(txt)
    for i in range(0,len(txt.lower().split('], ['))):
        if i ==0:
            c = txt.lower().split('], [')[i].replace("'","").replace(",","")
            b.append(c[2:])
        elif i == len(txt.lower().split('], ['))-1:
            c = txt.lower().split('], [')[i].replace("'","").replace(",","")
            b.append(c[:-2])
        else:
            b.append(txt.lower().split('], [')[i].replace("'","").replace(",",""))
    return(b)

# fonction pour nettoyer l'abstract
from nltk.corpus import stopwords
import unicodedata
from string import punctuation 

# netoyage de text
def preprocess_text(sen):
    stop = stopwords.words('english')
    punctuations=punctuation+"’”“‘…„—᾿‐–‑′•›‹⁄―‚→（）『』》《。↓↵'͞ʻʿ'"+'″￼'
    sen=str(sen)
    sen = sen[0:1000]
    sentence = sen.lower()
    sentence = str(sentence).strip()
    sentence = " ".join(sentence.split())
    for p in punctuations:
        sentence = sentence.replace(p," ")
    sentence = ''.join((c for c in unicodedata.normalize('NFD', sentence) if unicodedata.category(c) != 'Mn'))
    sentence = ''.join([i for i in sentence if not i.isdigit()])

    sentence = ' '.join([w for w in sentence.split() if len(w)>1])
    sentence = ' '.join([w for w in sentence.split() if len(w)<50])

    return sentence

def taille(colonne):
    return(len(str(colonne).split(" ")))

In [48]:
## apliquer netoyege et extaction de mots clés comme texte sur la dataframe
df['new_keywords']=df['keywords'].apply(keywords)
df['taille']=df['Abstract'].apply(taille)
df['txt']=df['Abstract'].astype(str) + df['new_keywords'].astype(str) 
df['net_text']=df['txt'].apply(preprocess_text)
df.drop(df.columns.difference(['Annee','Pays','taille','net_text','new_keywords']), 1, inplace=True)


,Annee,Pays,new_keywords,taille,net_text
0,2009,taiwan,"[algorithms, artificial intelligence, bayes th...",302,accurate grading for prostatic carcinoma in pa...
1,2014,taiwan,[atp binding cassette transporter subfamily g ...,130,background breast cancer resistant protein has...
2,2007,japan,"[algorithms, artificial intelligence, elastici...",173,we have developed thoracoscopic surgical navig...
3,2011,iran,"[adult, brain mapping methods, case control st...",187,schizophrenia is often considered as dysconnec...
4,2011,canada,"[algorithms, artificial intelligence, atherosc...",143,purpose three dimensional ultrasound us of the...
...,...,...,...,...,...
9986,2018,non renseigne,"[algorithms, humans, image interpretation comp...",355,automated methods for detecting clinically sig...
9987,2019,japan,"[anemia aplastic diagnosis, automation, diagno...",266,detection of dysmorphic cells in peripheral bl...
9988,2019,non renseigne,"[databases factual, dermoscopy methods, humans...",273,the presence of certain clinical dermoscopic f...
9989,2009,korea,"[algorithms, bile duct neoplasms diagnosis, bi...",196,cholangiocarcinoma cancer of the bile ducts is...


In [49]:
#######################################################################
## l'occurence des thermes dans chaque articles.

domaine=['drug design','detection','diagnostic','patient care']

AI_methodes=['artificial intelligence','support vector machine','imaging methods','decision tree','classification'
            ,'deep learning','robotic', 'logistic regression','neural networks','component analysis','data mining']
maladie=['cancer','schizophrenia',"diabetes","asthma","cardiovascular","alzheimer","neurodegenerative"]

# pour les des maladies
for e in maladie:
    df[e]=df.net_text.str.count(e)
### pour les methodes de l'intellegence artificielle
for e in AI_methodes:
    df[e]=df.net_text.str.count(e)

### pour domaine d'application
for e in domaine:
    df[e]=df.net_text.str.count(e)

# Parie III ) Traçage des graphes.

## graphes articles par payes

In [50]:
### preparations des datas pour le plot camembert main page payes articles
# cardiovascular'
df1p= df[df['cardiovascular'] >1]
df1p = df1p.groupby(['Pays']).cardiovascular.sum().reset_index()
df1p = df1p.dropna()
### preparation des labels pour pie graphes
df1p.loc[df1p['cardiovascular'] < 4 , 'Pays'] = 'Autre Pays'
df1p.loc[df1p['Pays'] == 'non renseigne' , 'Pays'] = 'Autre Pays'
df1p
l1=df1p.groupby('Pays')['cardiovascular'].sum()
l1.index
l1.values

# 'count_cancer'
df2p= df[df['cancer'] >1]
df2p = df2p.groupby(['Pays']).cancer.sum().reset_index()
df2p = df2p.dropna()
### preparation des labels pour pie graphes
df2p.loc[df2p['cancer'] < 50 , 'Pays'] = 'Autre Pays'
df2p.loc[df2p['Pays'] == 'non renseigne' , 'Pays'] = 'Autre Pays'
df2p
l2=df2p.groupby('Pays')['cancer'].sum()
l2.index
l2.values


# schizophrenia
df3p= df[df['schizophrenia'] >1]
df3p = df3p.groupby(['Pays']).schizophrenia.sum().reset_index()
df3p = df3p.dropna()
### preparation des labels pour pie graphes
df3p.loc[df3p['schizophrenia'] < 20 , 'Pays'] = 'Autre Pays'
df3p.loc[df3p['Pays'] == 'non renseigne' , 'Pays'] = 'Autre Pays'
l3=df3p.groupby('Pays')['schizophrenia'].sum()
l3.index
l3.values


# diabetes
df4p= df[df['diabetes'] >1]
df4p = df4p.groupby(['Pays']).diabetes.sum().reset_index()
df4p = df4p.dropna()
### preparation des labels pour pie graphes
df4p.loc[df4p['diabetes'] < 5 , 'Pays'] = 'Autre Pays'
df4p.loc[df4p['Pays'] == 'non renseigne' , 'Pays'] = 'Autre Pays'
l4=df4p.groupby('Pays')['diabetes'].sum()
l4.index
l4.values


# neurodegenerative
df5p= df[df['neurodegenerative'] >1]
df5p = df5p.groupby(['Pays']).neurodegenerative.sum().reset_index()
df5p = df5p.dropna()
### preparation des labels pour pie graphes
df5p.loc[df5p['neurodegenerative'] <2 , 'Pays'] = 'Autre Pays'
df5p.loc[df5p['Pays'] == 'non renseigne' , 'Pays'] = 'Autre Pays'
l5=df5p.groupby('Pays')['neurodegenerative'].sum()
l5.index
l5.values


# diabetes
df6p= df[df['asthma'] >1]
df6p = df6p.groupby(['Pays']).asthma.sum().reset_index()
df6p = df6p.dropna()
### preparation des labels pour pie graphes
df6p.loc[df6p['asthma'] < 2 , 'Pays'] = 'Autre Pays'
df6p.loc[df6p['Pays'] == 'non renseigne' , 'Pays'] = 'Autre Pays'
l6=df6p.groupby('Pays')['asthma'].sum()
l6.index
l6.values


array([19,  2, 10, 11,  3,  7,  5,  7,  2,  7,  2,  4,  3,  3,  2,  5, 23])

In [51]:

fig_p = make_subplots(2, 3, specs=[[{'type':'domain'},  {'type':'domain'},  {'type':'domain'}],
                                   [ {'type':'domain'}, {'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=['Maladie Cardiovasculaire','Maladie cancer','Maladie schizophrenia',
                                    'Maladie diabetes','Maladie neurodegenerative','Maladie asthme',])


fig_p.add_trace(go.Pie(labels=l1.index, values=l1.values), 1, 1)
fig_p.add_trace(go.Pie(labels=l2.index, values=l2.values), 1, 2)

fig_p.add_trace(go.Pie(labels=l3.index, values=l3.values), 1, 3)
fig_p.add_trace(go.Pie(labels=l4.index, values=l4.values), 2, 1)

fig_p.add_trace(go.Pie(labels=l5.index, values=l5.values), 2, 2)
fig_p.add_trace(go.Pie(labels=l6.index, values=l6.values), 2, 3)

fig_p.update_layout(title_text='Informations Génerales sur la publication des articles par Pays ')
fig_p.update_yaxes(automargin=True)
fig_p['layout'].update(width=1200, height=800, autosize=True)
fig_p.show()


## graphes pour chaque maladie et thermes IA

In [52]:
# creation de fonction pour extrait le nombre d'occurence des mots clés dans des articles pour chaque maladie.
from collections import defaultdict
def sentenses2vocabs(sents,th,n):
    fullcount=defaultdict(int)
    for x in sents:
        for w in x.split():
            fullcount[w]+=1
    vocabset = set(key for key, value in fullcount.items() if value >= th)
    vocab={}
    for i,x in enumerate(vocabset):
        if i>=n: ### les mots qu'ont plus que 100 fois dans le texte de tout les articles
            vocab[x]=i
    return vocab



In [61]:
# graphes cardiovascular


df1= df[df['cardiovascular'] >1]
# articles  cardiovascular
df1 = df1.groupby(['Annee']).cardiovascular.sum().reset_index()
# mots clés cardiovascular
df1_k=df[df['cardiovascular'] >2]
d_key1=sentenses2vocabs(df1_k['new_keywords'].sum(axis=0),th=2,n=150)
## domaine
domaine=['drug design','detection','diagnostic','patient care']
df11= df[df['cardiovascular'] >1]
domaine1=(df11['drug design'].sum(),df11['detection'].sum(),df11['diagnostic'].sum(),df11['patient care'].sum())
# algorithmes et methodes I A
AI_methodes
domaine2=[]
for e in AI_methodes:
    domaine2.append(df11[e].sum())


fig1 = make_subplots(rows=2, cols=2, subplot_titles=['Algorithmes et Methodes I.A', 'nombre articles cardiovascular', 'Mots clés','Domaine d application'])
fig1.add_bar(x=AI_methodes,y=domaine2, row=1, col=1)
fig1.add_scatter(x=df1['Annee'], y=df1['cardiovascular'], row=1,col=2)
fig1.add_bar(x=list(d_key1.keys()), y=list(d_key1.values()), row=2,col=1)
fig1.add_bar(x=domaine, y= domaine1, row=2, col=2)
fig1.update_yaxes(automargin=True)
fig1['layout'].update(width=1200, height=800, autosize=True)


Layout({
    'annotations': [{'font': {'size': 16},
                     'showarrow': False,
                     'text': 'Algorithmes et Methodes I.A',
                     'x': 0.225,
                     'xanchor': 'center',
                     'xref': 'paper',
                     'y': 1.0,
                     'yanchor': 'bottom',
                     'yref': 'paper'},
                    {'font': {'size': 16},
                     'showarrow': False,
                     'text': 'nombre articles cardiovascular',
                     'x': 0.775,
                     'xanchor': 'center',
                     'xref': 'paper',
                     'y': 1.0,
                     'yanchor': 'bottom',
                     'yref': 'paper'},
                    {'font': {'size': 16},
                     'showarrow': False,
                     'text': 'Mots clés',
                     'x': 0.225,
                     'xanchor': 'center',
                     'xref': 'paper',
           

In [60]:

# graphes cancer'
df2= df[df['cancer'] >1]
# articles cancer
df2 = df2.groupby(['Annee']).cancer.sum().reset_index()

# mots clés cancer
df2_k=df[df['cancer']>3]
d_key2=sentenses2vocabs(df2_k['new_keywords'].sum(axis=0),th=2,n=650)
## domaine
domaine=['drug design','detection','diagnostic','patient care']
df22= df[df['cancer'] >1]
domaine1=(df22['drug design'].sum(),df22['detection'].sum(),df22['diagnostic'].sum(),df22['patient care'].sum())
# algorithmes et methodes I A
AI_methodes
domaine2=[]
for e in AI_methodes:
    domaine2.append(df22[e].sum())


fig2 = make_subplots(rows=2, cols=2, subplot_titles=['Algorithmes et Methodes I.A','articles cancer','mots clés', 'domaine d application'])
fig2.add_bar(x=AI_methodes, y= domaine2, row=1, col=1)
fig2.add_scatter(x=df2['Annee'], y= df2['cancer'], row=1, col=2)
fig2.add_bar(x=list(d_key2.keys()), y=list(d_key2.values()), row=2,col=1)
fig2.add_bar(x=domaine, y=domaine1, row=2,col=2)
fig2.update_yaxes(automargin=True)
fig2['layout'].update(width=1200, height=800, autosize=True)


Layout({
    'annotations': [{'font': {'size': 16},
                     'showarrow': False,
                     'text': 'Algorithmes et Methodes I.A',
                     'x': 0.225,
                     'xanchor': 'center',
                     'xref': 'paper',
                     'y': 1.0,
                     'yanchor': 'bottom',
                     'yref': 'paper'},
                    {'font': {'size': 16},
                     'showarrow': False,
                     'text': 'articles cancer',
                     'x': 0.775,
                     'xanchor': 'center',
                     'xref': 'paper',
                     'y': 1.0,
                     'yanchor': 'bottom',
                     'yref': 'paper'},
                    {'font': {'size': 16},
                     'showarrow': False,
                     'text': 'mots clés',
                     'x': 0.225,
                     'xanchor': 'center',
                     'xref': 'paper',
                     'y': 

In [59]:
# Graphes schizophrenia.

# articles
df3= df[df['schizophrenia'] >3]
df3 = df3.groupby(['Annee']).schizophrenia.sum().reset_index()
# mots clés
df3_k=df[df['schizophrenia']>2]
d_key3=sentenses2vocabs(df3_k['new_keywords'].sum(axis=0),th=2,n=200)
## domaine
domaine=['drug design','detection','diagnostic','patient care']
df22= df[df['schizophrenia'] >1]
domaine1=(df22['drug design'].sum(),df22['detection'].sum(),df22['diagnostic'].sum(),df22['patient care'].sum())
# algorithmes et methodes I A
AI_methodes
domaine2=[]
for e in AI_methodes:
    domaine2.append(df22[e].sum())



fig3 = make_subplots(rows=2, cols=2, subplot_titles=['Algorithmes et Methodes I.A', 'articles schizophrenia','mots clés', 'domaine d application'])
fig3.add_bar(x=AI_methodes, y= domaine2, row=1, col=1)
fig3.add_scatter(x=df3['Annee'], y= df3['schizophrenia'], row=1, col=2)
fig3.add_bar(x=list(d_key3.keys()), y=list(d_key3.values()), row=2,col=1)
fig3.add_bar(x=domaine, y=domaine1, row=2,col=2)
fig3.update_yaxes(automargin=True)
fig3['layout'].update(width=1200, height=800, autosize=True)


Layout({
    'annotations': [{'font': {'size': 16},
                     'showarrow': False,
                     'text': 'Algorithmes et Methodes I.A',
                     'x': 0.225,
                     'xanchor': 'center',
                     'xref': 'paper',
                     'y': 1.0,
                     'yanchor': 'bottom',
                     'yref': 'paper'},
                    {'font': {'size': 16},
                     'showarrow': False,
                     'text': 'articles schizophrenia',
                     'x': 0.775,
                     'xanchor': 'center',
                     'xref': 'paper',
                     'y': 1.0,
                     'yanchor': 'bottom',
                     'yref': 'paper'},
                    {'font': {'size': 16},
                     'showarrow': False,
                     'text': 'mots clés',
                     'x': 0.225,
                     'xanchor': 'center',
                     'xref': 'paper',
                   

In [58]:
# graphe diabetes
df4= df[df['diabetes'] >= 1]
df4 = df4.groupby(['Annee']).diabetes.sum().reset_index()

# mots clés
df4_k=df[df['diabetes']>1]
d_key4=sentenses2vocabs(df4_k['new_keywords'].sum(axis=0),th=2,n=400)
## domaine
domaine=['drug design','detection','diagnostic','patient care']
df22= df[df['diabetes'] >1]
domaine1=(df22['drug design'].sum(),df22['detection'].sum(),df22['diagnostic'].sum(),df22['patient care'].sum())
# algorithmes et methodes I A
AI_methodes
domaine2=[]
for e in AI_methodes:
    domaine2.append(df11[e].sum())

fig4 = make_subplots(rows=2, cols=2, subplot_titles=['Algorithmes et Methodes I.A', 'articles diabetes', 'mots clés diabetes','domaine d application'])
fig4.add_bar(x=AI_methodes, y= domaine2, row=1, col=1)
fig4.add_scatter(x=df4['Annee'], y= df4['diabetes'], row=1, col=2)
fig4.add_bar(x=list(d_key4.keys()), y=list(d_key4.values()), row=2,col=1)
fig4.add_bar(x=domaine, y=domaine1, row=2,col=2)
fig4.update_yaxes(automargin=True)
fig4['layout'].update(width=1200, height=800, autosize=True)


Layout({
    'annotations': [{'font': {'size': 16},
                     'showarrow': False,
                     'text': 'Algorithmes et Methodes I.A',
                     'x': 0.225,
                     'xanchor': 'center',
                     'xref': 'paper',
                     'y': 1.0,
                     'yanchor': 'bottom',
                     'yref': 'paper'},
                    {'font': {'size': 16},
                     'showarrow': False,
                     'text': 'articles diabetes',
                     'x': 0.775,
                     'xanchor': 'center',
                     'xref': 'paper',
                     'y': 1.0,
                     'yanchor': 'bottom',
                     'yref': 'paper'},
                    {'font': {'size': 16},
                     'showarrow': False,
                     'text': 'mots clés diabetes',
                     'x': 0.225,
                     'xanchor': 'center',
                     'xref': 'paper',
               

In [57]:
# graphe  neurodegenerative
# articles

# mots clés
df5_k=df[df["neurodegenerative"]>1]
d_key5=sentenses2vocabs(df5_k['new_keywords'].sum(axis=0),th=2,n=100)
# articles
df5= df[df['neurodegenerative'] >= 1]
df5 = df5.groupby(['Annee']).neurodegenerative.sum().reset_index()
### domaine
domaine=['drug design','detection','diagnostic','patient care']
df22= df[df["neurodegenerative"] >1]
domaine1=(df22['drug design'].sum(),df22['detection'].sum(),df22['diagnostic'].sum(),df22['patient care'].sum())
# algorithmes et methodes I A
AI_methodes
domaine2=[]
for e in AI_methodes:
    domaine2.append(df11[e].sum())

fig5 = make_subplots(rows=2, cols=2, subplot_titles=['Algorithmes et Methodes I.A', 'Articles neurodegenerative', 'mot clés','domaine d application'])

fig5.add_bar(x=AI_methodes, y= domaine2, row=1, col=1)
fig5.add_scatter(x=df5['Annee'], y= df5['neurodegenerative'], row=1, col=2)
fig5.add_bar(x=list(d_key5.keys()), y=list(d_key5.values()), row=2,col=1)
fig5.add_bar(x=domaine, y=domaine1, row=2,col=2)
fig5.update_yaxes(automargin=True)
fig5['layout'].update(width=1200, height=800, autosize=True)


Layout({
    'annotations': [{'font': {'size': 16},
                     'showarrow': False,
                     'text': 'Algorithmes et Methodes I.A',
                     'x': 0.225,
                     'xanchor': 'center',
                     'xref': 'paper',
                     'y': 1.0,
                     'yanchor': 'bottom',
                     'yref': 'paper'},
                    {'font': {'size': 16},
                     'showarrow': False,
                     'text': 'Articles neurodegenerative',
                     'x': 0.775,
                     'xanchor': 'center',
                     'xref': 'paper',
                     'y': 1.0,
                     'yanchor': 'bottom',
                     'yref': 'paper'},
                    {'font': {'size': 16},
                     'showarrow': False,
                     'text': 'mot clés',
                     'x': 0.225,
                     'xanchor': 'center',
                     'xref': 'paper',
                

In [62]:
# asthma

# 'count_asthma'
df6=df[df['asthma']>1]
df6 = df6.groupby(['Annee']).asthma.sum().reset_index()
df6_k=df[df['asthma']>1]
d_key6=sentenses2vocabs(df6_k['new_keywords'].sum(axis=0),th=2,n=100)
### domaine
domaine=['drug design','detection','diagnostic','patient care']
df22= df[df["asthma"] >1]
domaine1=(df22['drug design'].sum(),df22['detection'].sum(),df22['diagnostic'].sum(),df22['patient care'].sum())
# algorithmes et methodes I A
AI_methodes
domaine2=[]
for e in AI_methodes:
    domaine2.append(df11[e].sum())


fig6 = make_subplots(rows=2, cols=2, subplot_titles=['articles asthma','Algorithmes et Methodes I.A','mots clés','domaine d application'])
fig6.add_scatter(x=df6['Annee'], y= df6['asthma'], row=1, col=1)
fig6.add_bar(x=AI_methodes, y=domaine2, row=1,col=2)
fig6.add_bar(x=list(d_key6.keys()), y=list(d_key6.values()), row=2,col=1)
fig6.add_bar(x=domaine, y=domaine1, row=2,col=2)
fig6.update_yaxes(automargin=True)
fig6['layout'].update(width=1200, height=800, autosize=True)


Layout({
    'annotations': [{'font': {'size': 16},
                     'showarrow': False,
                     'text': 'articles asthma',
                     'x': 0.225,
                     'xanchor': 'center',
                     'xref': 'paper',
                     'y': 1.0,
                     'yanchor': 'bottom',
                     'yref': 'paper'},
                    {'font': {'size': 16},
                     'showarrow': False,
                     'text': 'Algorithmes et Methodes I.A',
                     'x': 0.775,
                     'xanchor': 'center',
                     'xref': 'paper',
                     'y': 1.0,
                     'yanchor': 'bottom',
                     'yref': 'paper'},
                    {'font': {'size': 16},
                     'showarrow': False,
                     'text': 'mots clés',
                     'x': 0.225,
                     'xanchor': 'center',
                     'xref': 'paper',
                     'y': 

In [66]:
# graphes alzheimer

# mots clés
df7=df[df['alzheimer']>1]
d_key7=sentenses2vocabs(df7['new_keywords'].sum(axis=0),th=2,n=320)
# articles
df7 = df7.groupby(['Annee']).alzheimer.sum().reset_index()
### domaine
domaine=['drug design','detection','diagnostic','patient care']
df22= df[df['alzheimer'] >1]
domaine1=(df22['drug design'].sum(),df22['detection'].sum(),df22['diagnostic'].sum(),df22['patient care'].sum())
# algorithmes et methodes I A
AI_methodes
domaine2=[]
for e in AI_methodes:
    domaine2.append(df11[e].sum())

fig7 = make_subplots(rows=2, cols=2, subplot_titles=['Algorithmes et Methodes I.A','articles alzheimer', 'mots clés','domaine d appilcation'])
fig7.add_bar(x=AI_methodes, y= domaine2, row=1, col=1)
fig7.add_scatter(x=df7['Annee'], y= df7['alzheimer'], row=1, col=2)
fig7.add_bar(x=list(d_key7.keys()), y=list(d_key7.values()), row=2,col=1)
fig7.add_bar(x=domaine, y=domaine1, row=2,col=2)

fig7.update_yaxes(automargin=True)

fig7['layout'].update(width=1200, height=800, autosize=True)

Layout({
    'annotations': [{'font': {'size': 16},
                     'showarrow': False,
                     'text': 'Algorithmes et Methodes I.A',
                     'x': 0.225,
                     'xanchor': 'center',
                     'xref': 'paper',
                     'y': 1.0,
                     'yanchor': 'bottom',
                     'yref': 'paper'},
                    {'font': {'size': 16},
                     'showarrow': False,
                     'text': 'articles alzheimer',
                     'x': 0.775,
                     'xanchor': 'center',
                     'xref': 'paper',
                     'y': 1.0,
                     'yanchor': 'bottom',
                     'yref': 'paper'},
                    {'font': {'size': 16},
                     'showarrow': False,
                     'text': 'mots clés',
                     'x': 0.225,
                     'xanchor': 'center',
                     'xref': 'paper',
                     'y

## Graphe générale des articles des maladie en fonctions des années des therme IA et Pays

In [64]:
import plotly.express as px 
dfh=df.copy()
dfh['I.A'] = dfh.loc[dfh['cancer']>=0,AI_methodes].sum(axis=1)
dfh['Maladie'] = dfh.loc[dfh['I.A']>0,maladie].sum(axis=1)
dfh['I.A'].fillna(0, inplace=True)
dfh['Maladie'].fillna(0, inplace=True)
dfh=dfh[dfh['Maladie']>2]
fig_map = px.scatter(dfh, x="Annee", y="Pays", 
                 color="I.A", 
                 size="I.A",  
                 hover_data=maladie+AI_methodes) 
  


## nuage des mots dans tout les abstract

In [67]:
# github 
def plotly_wordcloud(text):
    wc = WordCloud(stopwords = set(STOPWORDS),
                   max_words = 200,
                   max_font_size = 100)
    wc.generate(text)
    
    word_list=[]
    freq_list=[]
    fontsize_list=[]
    position_list=[]
    orientation_list=[]
    color_list=[]

    for (word, freq), fontsize, position, orientation, color in wc.layout_:
        word_list.append(word)
        freq_list.append(freq)
        fontsize_list.append(fontsize)
        position_list.append(position)
        orientation_list.append(orientation)
        color_list.append(color)
        
    # get the positions
    x=[]
    y=[]
    for i in position_list:
        x.append(i[0])
        y.append(i[1])
            
    # get the relative occurence frequencies
    new_freq_list = []
    for i in freq_list:
        new_freq_list.append(i*100)
    new_freq_list
    
    trace = go.Scatter(x=x, 
                       y=y, 
                       textfont = dict(size=new_freq_list,
                                       color=color_list),
                       hoverinfo='text',
                       hovertext=['{0}{1}'.format(w, f) for w, f in zip(word_list, freq_list)],
                       mode='text',  
                       text=word_list
                      )
    
    layout = go.Layout({'xaxis': {'showgrid': False, 'showticklabels': False, 'zeroline': False},
                        'yaxis': {'showgrid': False, 'showticklabels': False, 'zeroline': False}})
    
    fig = go.Figure(data=[trace], layout=layout)
    
    return fig
### application sur tout la dataframe
T=[]
for k in range(len(df)):
    T.append(df['net_text'][k])
    
figwc=plotly_wordcloud(str(T))
figwc['layout'].update(width=1200, height=800, autosize=True)


Layout({
    'autosize': True,
    'height': 800,
    'template': '...',
    'width': 1200,
    'xaxis': {'showgrid': False, 'showticklabels': False, 'zeroline': False},
    'yaxis': {'showgrid': False, 'showticklabels': False, 'zeroline': False}
})

# Pratie VI ) Configuration Dash BoostRAP et construire pages HTML

In [ ]:

app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
    "background-color": "#C0C0C0",
}

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}
sidebar = html.Div(
    [
        html.H1("Projet Dashbord M1 SID", className="display-4"),
        html.Hr(),
        html.P(
            "la contribution de I.A dans les maladies incurables", className="lead"
        ),
        dbc.Nav(
            [
                dbc.NavLink("Premieres Graphes et Analyse", href="/", active="exact"),
                dbc.NavLink("Cardiovasculaire & I.A ", href="page-1", active="exact"),
                dbc.NavLink("Cancer & I.A", href="/page-2", active="exact"),
                dbc.NavLink("Schizophrenia & I.A", href="/page-3", active="exact"),
                dbc.NavLink("Diabetes  & I.A", href="/page-4", active="exact"),
                dbc.NavLink("Deurodegenerative & I.A ", href="/page-5", active="exact"),
                dbc.NavLink("Asthma & I.A ", href="/page-6", active="exact"),
                dbc.NavLink("Alzheimer & I.A ", href="/page-7", active="exact"),
                dbc.NavLink("Algorithmes I.A et Maladies", href="/page-8", active="exact"),
                dbc.NavLink("Nuage des mots totale", href="/page-9", active="exact"),
            ],
            vertical=True,
            pills=True,
        ),
    ],
    style=SIDEBAR_STYLE,
)

content = html.Div(id="page-content", style=CONTENT_STYLE)

app.layout = html.Div([dcc.Location(id="url"), sidebar, content])


In [ ]:
##### main page
p_g = html.P([html.Div(children=[
    # All elements from the top of the page
    html.Div([
            html.H1(children='Graphes des publications des articles des maladies incurables selon les pays'),
            dcc.Graph(id='graph1',figure=fig_p
            ),  
        ])
   ])
])

############
px1 = html.P([html.Div(children=[
    # All elements from the top of the page
    html.Div([
            html.H1(children='Cardiovascular diseas'),
            dcc.Graph(id='graph1',figure=fig1
            ),  
        ])
   ])
])

###############
px2 = html.P([html.Div(children=[
    # All elements from the top of the page
    html.Div([
            html.H1(children='Cancer'),
            dcc.Graph(id='graph1',figure=fig2
            ),  
        ])
   ])
])
######
px3 = html.P([html.Div(children=[
    # All elements from the top of the page
    html.Div([
            html.H1(children='Schizophrenia'),
            dcc.Graph(id='graph1',figure=fig3
            ),  
        ])
   ])
])

#####
px4 = html.P([html.Div(children=[
    # All elements from the top of the page
    html.Div([
            html.H1(children='Diabetes'),
            dcc.Graph(id='graph1',figure=fig4
            ),  
        ])
   ])
])

#####
px5 = html.P([html.Div(children=[
    # All elements from the top of the page
    html.Div([
            html.H1(children='Neurodegenerative'),
            dcc.Graph(id='graph1',figure=fig5
            ),  
        ])
   ])
])


######
px6 = html.P([html.Div(children=[
    # All elements from the top of the page
    html.Div([
            html.H1(children='Asthma'),
            dcc.Graph(id='graph1',figure=fig6
            ),
        ])
   ])
])


#######
px7 = html.P([html.Div(children=[
    # All elements from the top of the page
    html.Div([
            html.H1(children='Alzheimer'),
            dcc.Graph(id='graph1',figure=fig7
            ),
        ])
   ])
])
#######
px8 = html.P([html.Div(children=[
    # All elements from the top of the page
    html.Div([
            html.H1(children='Graphe générale des articles des maladie en fonctions des années, des algorithmes I.A et Pays'),
            dcc.Graph(id='graph1',figure=fig_map
            ),
        ])
   ])
])

#######
px9 = html.P([html.Div(children=[
    # All elements from the top of the page
    html.Div([
            html.H1(children='Nuage des mots dans notre scrapping abstracts'),
            dcc.Graph(id='graph1',figure=figwc
            ),
        ])
   ])
])


In [ ]:
# 5 Run app
@app.callback(Output("page-content", "children"), [Input("url", "pathname")])
def render_page_content(pathname):
    if pathname == "/":
        return p_g
    elif pathname == "/page-1":
        return px1
    elif pathname == "/page-2":
        return px2
    elif pathname == "/page-3":
        return px3
    elif pathname == "/page-4":
        return px4
    elif pathname == "/page-5":
        return px5
    elif pathname == "/page-6":
        return px6
    elif pathname == "/page-7":
        return px7
    elif pathname == "/page-8":
        return px8
    elif pathname == "/page-9":
        return px9
        
        
    # If the user tries to reach a different page, return a 404 message
    return dbc.Jumbotron(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ]
    )

if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Mar/2021 04:12:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Mar/2021 04:12:51] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [20/Mar/2021 04:12:51] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [20/Mar/2021 04:12:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Mar/2021 04:12:51] "GET /_dash-component-suites/dash_core_components/async-plotlyjs.v1_15_0m1611086576.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Mar/2021 04:12:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Mar/2021 04:12:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Mar/2021 04:12:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Mar/2021 04:12:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Mar/2021 04:12:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Mar/2021 04:13:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Feb/2021 22:21:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2021 22:21:44] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2021 22:21:44] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2021 22:21:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2021 22:21:46] "POST /_dash-update-component HTTP/1.1" 200 -
